In [7]:
import numpy as np
np.random.seed(0)
from functions import *
#below is saved for my own reminder of what each setting does

"""init_nuc_num = 10000 # Initial number of each nucleobase
cleav_prop = 0.4 # Probability of breaking bonds in unstructured regions
cleav_prop_struct = 0.02 # Probability of breaking bonds in structured regions
length_threshold = 10 # Strands longer than this we will checked for structure
n_iterations = 100 # Number of iterations
progress_report_freq  = 10 # Number of iterations"""

#below are the variables we manipulate
init_nuc_num = 10000 
cleav_prop = 0.4
cleav_prop_struct = 0.03 
length_threshold = 10 
n_iterations = 100
progress_report_freq  = 100


#below are string versions of the variables used - this is written to our output file to help us track what settings produced the sequences
# the * is put before every non-sequence item to give us a way to ignore these lines when processing data
init_nuc_num_str = "*init_nuc_num = " + str(init_nuc_num) + "\n"
cleav_prop_str = "*cleav_prop = " + str(cleav_prop) + "\n"
cleav_prop_struct_str = "*cleav_prop_struct = " + str(cleav_prop_struct) + "\n"
length_threshold_str = "*length_threshold = " + str(length_threshold) + "\n"
n_iterations_str = "*n_iterations = " + str(n_iterations) + "\n"
progress_report_freq_str = "*progress_report_freq = " + str(progress_report_freq) + "\n"
settings_used = init_nuc_num_str + cleav_prop_str + cleav_prop_struct_str +  \
    length_threshold_str +  n_iterations_str + progress_report_freq_str 

# A general note: Here we use numpy array of dtype=object because Python
# can support arbitrarily large integers. In contrast, NumPy int64 type
# will overflow for long sequences
nucleotide_list = np.array([1, 2, 3, 4] * init_nuc_num, dtype=object)
# Mapping from integer numbers to nucleobase names
mapping = {"1": "A", "2": "G", "3": "C", "4": "U"}

# Initial Phase 1
# Shuffle the nucleobases, then pair them
# We represent a nucleic acid strand by a sequence of integers (e.g. 11 is AA)
# So multiply a number by 10 and to another number to create a dimer
np.random.shuffle(nucleotide_list)
size = nucleotide_list.size
nucleotide_list = nucleotide_list[:size//2] + 10 * nucleotide_list[size//2:]

# Initial Phase 3
# Randomly break dimer bonds to create a set of monomers and dimers
cleave = np.random.random(nucleotide_list.size) > cleav_prop
cleaved = nucleotide_list[cleave]
uncleaved = nucleotide_list[~cleave]

cleaved_1 = cleaved//10
cleaved_2 = cleaved%10

nucleotide_list = np.concatenate((uncleaved, cleaved_1, cleaved_2))

# Loop over the number of iterations
file_to_write_to = open("test_file_1.txt", "w")
file_to_write_to.write("*SETTINGS USED IN THE BELOW ITERATION: " + "\n" + settings_used + "\n")
for it in range(1, n_iterations + 1):
    print(it)
    #file_to_write_to.write("\n")
    

    # Phase 1: Pair nucleotide strands
    # Shuffle list of nucleotide strands
    np.random.shuffle(nucleotide_list)

    # Take the first half of the strands and calculate their lengths
    size = nucleotide_list.size
    # Use dtype=object to keep arbitrary integer prevision
    order = order_of_mag(nucleotide_list[:size//2]).astype(object)

    # If we have an even number, pair all of them
    if size%2 == 0:
        nucleotide_list = nucleotide_list[:size//2] + 10**order * nucleotide_list[size//2:]
    # If we have an odd number, do not pair last strand
    else:
        nucleotide_list_temp = nucleotide_list[:size//2] + 10**order * nucleotide_list[size//2:-1]
        nucleotide_list = np.hstack((nucleotide_list_temp, nucleotide_list[-1]))

    # Phase 2: Determine folded structures and randomly break bonds in long strands
    order = order_of_mag(nucleotide_list).astype(object)
    mono = nucleotide_list[order == 1]
    short = nucleotide_list[np.logical_and(order > 1, order < length_threshold)]
    long = nucleotide_list[order >= length_threshold]

    long = break_long(long, cleav_prop, cleav_prop_struct, mapping)

    # Phase 3: Randomly break bonds in short strands
    short = break_short(short, cleav_prop)

    nucleotide_list = np.concatenate((mono, short, long))


    # Print progress report: show sequences of 10 longest strands
    if it%progress_report_freq==0:
        print("\nRESULTS\n")

        summary = np.sort(nucleotide_list)[::-1] #to get all lengths, remove [::-1][-:10]
        print(summary)
        summary_s = convert_int_to_str_seq(summary, mapping)
        file_to_write_to.write("\n")
        for s in summary_s:
            print(s)
            file_to_write_to.write(s + "\n")


file_to_write_to.close()

generate_sequence_length_histogram('test_file_1.txt')

 #add histogram of length distribution - #1 thing to add
 #add function to reset simulation in middle of a length of iteration --> how sensitive is the data 
 #coming out to my parameters? what is the impact of the randon nums on what is observed?
 #--> use of seed ensures that all random nums are the same, ensures observed diffs are bc of 
 # parameter changes, not random computation changes
 #if successful, s.s.v should be the same regardless of seed - no variation in histogram
 #work on a way to save parameters and pick back up at the same seed (state)
 #monte carlo simulations

#how to do
 #run random nums n iterations, plot values that come out, we will observe a bunch of noise
 #if we run it w/o a seed and run n interations, patterns will be diff
 #but, if we start w same seed, pattern will be identical, but both will still look like noise
 #the goal is to produce a method for repeatability 


 #invrstigate seeds and usage in random num generators




#this assumes equal probability of creation of sequences - equal stability --> 
#perhaps an idea for further developments
            


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100

RESULTS

[4124343133442 232121213321 22313434211 ... 1 1 1]
UAGUCUCACCUUG
GCGAGAGACCGA
GGCACUCUGAA
AUUAUAGAAAA
AACUGCUUAC
CCUGGUAUU
GCUGUAGCG
GAAGAUAAU
AGAAUUUCC
UCGGAUUG
CCUAAUGU
CGAUGGGA
GCGGCUCA
GGAAUACC
GAGCCCUG
AGAUUGAC
UUUUUCC
UGGUCAG
UGACCUG
CAUGGUU
CACGUUC
GGCGCUA
GGGAUAG
GAUUGGC
GAAGGAA
ACUUGUU
ACUCAGU
ACCAGCG
ACCAGGG
AGAUGUU
AAUACGC
UUCAGA
UUGUAG
UUGCAA
UUGGAA
UUGACU
UCUGAG
UCCUCC
UCGUAA
UCGGUA
UCGGGG
UCGAUC
UCGAAC
UCAGUG
UCAACC
UGCGUC
UGCGGC
UGAGCU
UGAAUG
UAUCGG
UAUAGC
UACCAG
UAGUGA
UAGCCC
UAGGGG
UAAGGG
CUUACA
CUGCAC
CUGACU
CUACCG
CCUUCC
CCUGUG
CCCCCA
CCCACC
CCGUGC
CCGUAA
CCGAGU
CCACUC
CGGGGC
CAUGCG
CACUCU
CACUGA
CACAAG
CAGUUG
CAGCCA
CAAACA
GUUGCA
GUCAGA
GUAACU
GCCCUA
GCCCCA
GCCGGC
GCCGG

NameError: name 'peepee' is not defined